<a href="https://colab.research.google.com/github/itaewonflow/lecture-ai-text-focus/blob/main/AI_Text_summarization_Hangule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- github : https://github.com/itaewonflow
- Explain : https://decadepivot.com/
---


# 트랜스포머 모형을 이용한 [한글] 문서 요약

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

## 한글 문서 요약

- 한글 문서를 요약하는 것은 기본적으로 영어 문서와 동일하다. 가장 중요한 차이는 한글 말뭉치에 대해 학습된 토크나이저와 사전학습 언어모델이 있어야 한다는 것이다.

In [8]:
text = "인공지능 또는 AI는 인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현하려는 컴퓨터 과학의 세부분야 중 하나이다. 정보공학 분야에 있어 하나의 인프라 기술이기도 하다.인간을 포함한 동물이 갖고 있는 지능 즉, natural intelligence와는 다른 개념이다. 인간의 지능을 모방한 기능을 갖춘 컴퓨터 시스템이며, 인간의 지능을 기계 등에 인공적으로 시연(구현)한 것이다. 일반적으로 범용 컴퓨터에 적용한다고 가정한다. 이 용어는 또한 그와 같은 지능을 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 과학 기술 분야를 지칭하기도 한다."
preprocess_text = text.strip().replace("\n","")

###KoBART
- 사전학습 언어모델: gogamaza/kobart-summarization
- 토크나이저와 모형: PreTrained ToKenizerFast와 BartForConditionalGeneration사용

In [9]:
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt")
summary_ids = model.generate(tokenized_text,
                             num_beams=4, # beam의 길이
                             no_repeat_ngram_size=3, #동어 반복을 피하기 위해 사용
                             min_length=10,  #요약문의 최소 토큰 수
                             max_length=150,  #요약문의 최대 토큰 수
                             early_stopping=True) #EOS 토큰을 만나면 종료
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


인간의 지능 지능을 모방한 기능을 갖춘 컴퓨터 시스템이며, 인간의 지능을 기계 등에 인공적으로 시연(구현)한 것이다. 일반적으로 범용 컴퓨터에 적용한다고 가정한다.


###mT5
- 사전학습 언어모델: csebuetnlp/mT5_multilingual_XLSum
- 토크나이저와 모형: AutoTokenizer와 AutoModelForSeq2SeqLM

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt")
summary_ids = model.generate(tokenized_text,
                             num_beams=4, # beam의 길이
                             no_repeat_ngram_size=2, #동어 반복을 피하기 위해 사용
                             min_length=10,  #요약문의 최소 토큰 수
                             max_length=150,  #요약문의 최대 토큰 수
                             early_stopping=True) #EOS 토큰을 만나면 종료
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

인공지능(AI)는 인간의 지능을 모방한 컴퓨터 시스템이다.
